In [2]:
import random
import numpy as np
from deap import creator, base, tools, algorithms

In [3]:
creator.create('FitnessMax', base.Fitness, weights=(1,))
creator.create('Individual', list, fitness=creator.FitnessMax)

In [4]:
toolbox = base.Toolbox()

In [5]:
toolbox.register('attr_bool', random.randint, 0, 1)
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.attr_bool, n=20)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

In [6]:
def evalOneMax(individual):
#     return (sum(individual),)
    index = len(individual) // 2
    fitness = sum(individual[index:]) - sum(individual[:index])
    return (fitness, )

In [7]:
toolbox.register('evaluate', evalOneMax)
toolbox.register('mate', tools.cxTwoPoint)
toolbox.register('mutate', tools.mutFlipBit, indpb=0.05)
toolbox.register('select', tools.selTournament, tournsize=3)

In [8]:
NGEN = 100
population = toolbox.population(n=300)
for gen in range(NGEN):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1)
    print(offspring[3])
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = np.array([fit])
    print(offspring[3].fitness.values)
    population = toolbox.select(offspring, k=len(population))
top10 = tools.selBest(population, k=10)

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0]
(array([0.]),)
[0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0]
(array([0.]),)
[0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0]
(array([2.]),)
[0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0]
(array([4.]),)
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0]
(array([4.]),)
[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
(array([7.]),)
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1]
(array([7.]),)
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
(array([9.]),)
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
(array([9.]),)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
(array([9.]),)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
(array([10.]),)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
(array([9.]),)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
(array([10.]),)
[0, 0, 0, 

In [9]:
population = toolbox.population(n=300)
result = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=100, verbose=False)

In [10]:
best_individual = tools.selBest(population, k=1)[0]
best_individual, evalOneMax(best_individual)

([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], (10,))

In [11]:
top10[0], evalOneMax(top10[0])

([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], (10,))

## Knapsack

In [15]:
import random
import numpy as np
from deap import creator, base, tools, algorithms

In [18]:
creator.create('Fitness', base.Fitness, weights=(-1.0, 1.0))
creator.create('Individual', set, fitness=creator.Fitness)

In [21]:
N_ITEMS = 20
MAX_ITEM = 50
MAX_WEIGHT = 50
IND_INIT_SIZE = 5

In [22]:
items = {}
for i in range(N_ITEMS):
    items[i] = (random.randint(1, 10), random.uniform(0, 100))

In [24]:
toolbox = base.Toolbox()

toolbox.register('attr_item', random.randrange, N_ITEMS)

toolbox.register('individual', tools.initRepeat, creator.Individual,
                 toolbox.attr_item, IND_INIT_SIZE)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

In [47]:
def evalKnapsack(individual):
    weight = sum(items[item][0] for item in individual)
    value = sum(items[item][1] for item in individual)
    if len(individual) > MAX_ITEM or weight > MAX_WEIGHT:
        return 10000, 0 # Overweighted bags are dominated
    return weight, value

In [30]:
def cxSet(ind1, ind2):
    return ind1 & ind2, ind1 ^ ind2

In [31]:
def mutSet(individual):
    if random.random() < 0.5:
        if len(individual) > 0:
            individual.remove(random.choice(sorted(tuple(individual))))
    else:
        individual.add(random.randrange(N_ITEMS))
    return individual, 

In [48]:
toolbox.register('evaluate', evalKnapsack)
toolbox.register('mate', cxSet)
toolbox.register('mutate', mutSet)
toolbox.register('select', tools.selNSGA2)

In [87]:
random.seed(64)
NGEN = 150
MU = 50
LAMBDA = 100
CXPB = 0.7
MUTPB = 0.2

In [73]:
mat = np.zeros((len(items)+1, MAX_WEIGHT+1))
mat[0,:] = mat[:,0] = 0
for i in range(len(items)):
    for j in range(MAX_WEIGHT):
        if j+1 - items[i][0] >= 0:
            mat[i+1,j+1] = max(mat[i,j+1], items[i][1] + mat[i,j+1 - items[i][0]])
        else:
            mat[i+1,j+1] = mat[i,j+1]

In [74]:
mat[len(items),MAX_WEIGHT]

754.8369115683012

In [95]:
sum(items[item][0] for item in hof.items[-1]), sum(items[item][1] for item in hof.items[-1])

(49, 750.3699904724358)

In [94]:
hof.items[-1]

{1, 2, 3, 4, 6, 7, 8, 10, 13, 14, 15, 18}

In [92]:
pop = toolbox.population(n=MU)
hof = tools.ParetoFront()

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register('avg', np.mean, axis=0)
stats.register('std', np.std, axis=0)
stats.register('min', np.min, axis=0)
stats.register('max', np.max, axis=0)

In [93]:
algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, 
                          stats, halloffame=hof)

gen	nevals	avg                        	std                      	min                      	max                        
0  	50    	[ 22.7        214.70463351]	[ 4.75920161 64.96670564]	[11.         39.88973863]	[ 40.         365.55756961]
1  	87    	[  7.14       117.17447162]	[  8.72928405 122.2048343 ]	[0. 0.]                  	[ 28.         365.55756961]
2  	91    	[ 3.18       72.04216147]  	[  6.12434486 124.16352979]	[0. 0.]                  	[ 23.         374.23520053]
3  	86    	[ 1.84       52.97877194]  	[  3.59644269 100.73013778]	[0. 0.]                  	[ 13.         413.38002849]
4  	92    	[ 2.5        70.93150155]  	[  4.10974452 114.55858691]	[0. 0.]                  	[ 13.         413.38002849]
5  	91    	[ 1.8        58.90941993]  	[  3.79473319 112.74889916]	[0. 0.]                  	[ 13.         413.38002849]
6  	90    	[ 1.64       66.42806437]  	[  3.29702897 107.11016098]	[0. 0.]                  	[ 13.         413.38002849]
7  	90    	[ 1.42       53.70771325]

([{1, 2, 3, 4, 6, 7, 8, 10, 13, 14, 15, 18},
  {1, 2, 3, 4, 6, 7, 8, 10, 13, 14, 15, 18},
  {1, 2, 3, 4, 6, 7, 8, 10, 13, 14, 15, 18},
  set(),
  set(),
  {10},
  {10},
  {8, 10},
  {8, 10},
  {8, 10, 13},
  {2, 3, 4, 7, 8, 10, 13, 14},
  {2, 7, 8, 10, 13},
  {4, 8, 10, 13},
  {3, 4, 8, 10, 13, 14, 15},
  {3, 4, 6, 8, 10, 13, 14, 15, 18},
  {2, 3, 4, 8, 10, 13, 15},
  {2, 3, 4, 7, 8, 9, 10, 13, 14, 15},
  {2, 8, 10, 13, 14},
  {2, 8, 10, 13, 14},
  {2, 8, 10, 13, 14},
  {3, 8, 10, 13, 15},
  {2, 3, 8, 10, 13, 15},
  {3, 4, 8, 10, 13, 14, 15, 18},
  {2, 3, 4, 6, 8, 10, 13, 14, 15, 18},
  {2, 3, 4, 7, 8, 10, 13, 14, 15, 18},
  {2, 3, 4, 7, 8, 10, 13, 14, 15, 18},
  {2, 3, 4, 7, 8, 10, 13, 14, 15, 18},
  {2, 3, 4, 6, 7, 8, 10, 11, 13, 14, 15, 18},
  {2, 3, 4, 6, 7, 8, 10, 11, 13, 14, 15, 18},
  {2, 3, 4, 7, 8, 9, 10, 13, 14, 15, 18},
  {2, 3, 4, 6, 8, 9, 10, 13, 14, 15, 18},
  {2, 3, 4, 8, 9, 10, 13, 14, 15},
  {3, 4, 8, 10, 13, 14, 15, 18},
  {2, 4, 7, 8, 10, 13, 14},
  {3, 4, 8, 10, 13,